In [203]:
import numpy as np
import pandas as pd
pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

In [340]:
operating_point_file = "operating_point_version2.csv"
op = pd.read_csv(operating_point_file, header=0, index_col = [0])
op = op.drop(['region']).astype(float)

In [341]:
op

,M1,M2,M3,M4,M5,M6,M7,M8
ids,105.6u,105.6u,-105.6u,-105.6u,-646.7u,646.7u,211.2u,10.0u
vgs,745.0m,745.0m,-758.2m,-758.2m,-765.0m,615.1m,615.1m,615.1m
vds,1.2,1.2,-758.2m,-765.0m,-1.3,1.2,505.0m,615.1m
vdsat,230.3m,230.4m,-241.9m,-241.9m,-247.9m,204.1m,203.3m,198.5m
vbs,-505.0m,-505.0m,0.0,0.0,0.0,0.0,0.0,0.0
vth,498.0m,498.1m,-509.7m,-509.7m,-509.6m,380.8m,381.9m,388.8m
vgsteff,246.3m,246.4m,249.2m,249.2m,256.0m,232.8m,231.7m,224.8m
gm,786.3u,786.3u,742.9u,743.1u,4.4m,5.1m,1.7m,82.2u
gds,4.0u,4.0u,2.5u,2.5u,11.1u,13.5u,7.6u,289.0n
gmb,144.4u,144.4u,241.5u,241.5u,1.4m,1.1m,374.8u,18.2u


In [343]:
# version 1
# ota_param = {"CL": 20e-12, "RL": 10e6, "R1": 100e9, "R2": 100e6, "Cc": 3.3e-12, "Rc": 1700}
# version 2
ota_param = {"CL": 20e-12, "RL": 10e6, "R1": 100e9, "R2": 100e6, "Cc": 10.0e-12, "Rc": 500}
# version 3 
# ota_param = {"CL": 20e-12, "RL": 10e6, "R1": 100e9, "R2": 100e6, "Cc": 5.0e-12, "Rc": 1000}
# presized
# ota_param = {"CL": 880e-12, "RL": 10e6, "R1": 100e9, "R2": 100e6, "Cc": 330e-12, "Rc": 740}

In [344]:
ota_perf = {}

In [345]:
# DC gains (still need to add loading of feedback network)
ota_perf["A1"] = op["M2"]["gm"]/(op["M2"]["gds"]+op["M3"]["gds"])
ota_perf["A2"] = op["M5"]["gm"]/(op["M5"]["gds"]+op["M6"]["gds"]+1/ota_param["RL"])
ota_perf["ADC_dB"] = 20*np.log10(ota_perf["A1"] * ota_perf["A2"])
# fu
ota_perf["fu"] = op["M1"]["gm"]/2/np.pi/ota_param["Cc"]
# second stage w/ Miller
ota_perf["f2ndstage"] = op["M5"]["gm"]/2/np.pi/ota_param["CL"]
ota_perf["fzero"] = 1/2/np.pi/ota_param["Cc"]/(1/op["M5"]["gm"]-ota_param["Rc"])
ota_perf["fpoleRc"] = 1/2/np.pi/(ota_param["Rc"])/op["M5"]["cgs"]
# current mirror
ota_perf["fmir"] = op["M3"]["gm"]/2/np.pi/(op["M3"]["cgs"]+op["M3"]["cbd"]+op["M4"]["cgs"])
# phase shifts
ota_perf["phase_shift_2ndstage"] = (- np.arctan(ota_perf["fu"]/ota_perf["f2ndstage"]))/np.pi*180
ota_perf["phase_shift_zero"] = (- np.arctan(ota_perf["fu"]/ota_perf["fzero"]))/np.pi*180
ota_perf["phase_shift_poleRc"] = (- np.arctan(ota_perf["fu"]/ota_perf["fpoleRc"]))/np.pi*180
ota_perf["phase_shift_mirror"] = (- np.arctan(ota_perf["fu"]/ota_perf["fmir"])
                  + np.arctan(ota_perf["fu"]/(2*ota_perf["fmir"])))/np.pi*180
# phase margin
ota_perf["PM_deg2"] = (90 + ota_perf["phase_shift_2ndstage"] + ota_perf["phase_shift_zero"] 
                       + ota_perf["phase_shift_poleRc"] + ota_perf["phase_shift_mirror"])

In [346]:
ota_perf_df = pd.Series(ota_perf)
ota_perf_df

A1                      120.1
A2                      179.5
ADC_dB                   86.7
fu                      12.5M
f2ndstage               35.3M
fzero                  -57.9M
fpoleRc                 49.5M
fmir                    51.7M
phase_shift_2ndstage    -19.5
phase_shift_zero         12.2
phase_shift_poleRc      -14.2
phase_shift_mirror       -6.7
PM_deg2                  61.8
dtype: float64

In [347]:
# print(ota_perf_df.to_markdown(floatfmt=".2f"))

In [348]:
ota_model = {}

In [349]:
ota_model["gm1"] = op["M1"]["gm"]
ota_model["Ro1"] = 1/(op["M2"]["gds"]+op["M3"]["gds"])
ota_model["Co1"] = op["M2"]["cbd"]+op["M3"]["cbd"]+op["M5"]["cgs"]
ota_model["gm2"] = op["M5"]["gm"]
ota_model["Ro2"] = 1/(op["M5"]["gds"]+op["M6"]["gds"])
ota_model["Co2"] = op["M5"]["cbd"]+op["M6"]["cbd"]
ota_model["Rmir"] = 1/(op["M3"]["gm"]+op["M3"]["gds"])
ota_model["Cmir"] = op["M3"]["cgs"]+op["M3"]["cbd"]+op["M4"]["cgs"]

In [350]:
ota_model_df = pd.Series(ota_model)
ota_model_df

gm1     786.3u
Ro1     152.7k
Co1       6.5p
gm2       4.4m
Ro2      40.6k
Co2     384.7f
Rmir      1.3k
Cmir      2.3p
dtype: float64